# Hello World of Predict Store Sales

This example is near replication of sample code published by DataBriefing for Kaggle Rossmann Store Sales competition. 

[Kaggle.com Rossmann Store Sales](https://www.kaggle.com/c/rossmann-store-sales/).

[Original DataBriefing Code](https://github.com/databriefing/article-notebooks/tree/master/rossmann).
[https://github.com/databriefing/article-notebooks/tree/master/rossmann](https://github.com/databriefing/article-notebooks/tree/master/rossmann).



In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
data = pd.read_csv('./data/train.csv')

In [ ]:
data.head()

In [ ]:
data.describe()

In [ ]:
data.dtypes

In [ ]:
data.StateHoliday.unique()

In [ ]:
# data.StateHoliday = data.StateHoliday.astype(str)

In [ ]:
data.StateHoliday.unique()

In [ ]:
# List the number of unique values for each colum
# Apply the function to each column (axis=0)

def count_unique(column):
    return len(column.unique())

data.apply(count_unique, axis=0).astype(np.int32)

In [ ]:
data.isnull().any()

In [ ]:
# Filter data for store 150 and plot sales data for first 365 days
store_data = data[data.Store==150].sort_values('Date')
plt.figure(figsize=(20, 10))  # Set figsize to increase size of figure
plt.plot(store_data.Sales.values[:365])

In [ ]:
plt.figure(figsize=(20, 10))
plt.scatter(x=store_data[data.Open==1].Promo, y=store_data[data.Open==1].Sales, alpha=0.1)
plt.xlabel('Promo')
plt.ylabel('Sales')

## Transformations

In [ ]:
transformed_data = data.drop(['Store', 'Date', 'Customers'], axis=1)

In [ ]:
transformed_data.head()

In [ ]:
transformed_data = pd.get_dummies(transformed_data, columns=['DayOfWeek', 'StateHoliday'])

In [ ]:
X = transformed_data.drop(['Sales'], axis=1).values
y = transformed_data.Sales.values
print("The training dataset has {} examples and {} features.".format(X.shape[0], X.shape[1]))

## Building and cross-validating a model

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
from sklearn import cross_validation as cv

In [ ]:
lr = LinearRegression()
kfolds = cv.KFold(X.shape[0], n_folds=4, shuffle=True, random_state=42)
scores = cv.cross_val_score(lr, X, y, cv=kfolds)

In [ ]:
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()))

## Visualizing predictions

We'll single out store 150 again and train our model on every store __except__ store 150 and then predict sales for store 150.

Remember: __Always__ use different data for training and predicting.

In [ ]:
lr = LinearRegression()
X_store = pd.get_dummies(data[data.Store!=150], columns=['DayOfWeek', 'StateHoliday']).drop(['Sales', 'Store', 'Date', 'Customers'], axis=1).values
y_store = pd.get_dummies(data[data.Store!=150], columns=['DayOfWeek', 'StateHoliday']).Sales.values
lr.fit(X_store, y_store)
y_store_predict = lr.predict(pd.get_dummies(store_data, columns=['DayOfWeek', 'StateHoliday']).drop(['Sales', 'Store', 'Date', 'Customers'], axis=1).values)

In [ ]:
plt.figure(figsize=(20, 10))  # Set figsize to increase size of figure
plt.plot(store_data.Sales.values[:365], label="ground truth")
plt.plot(y_store_predict[:365], c='r', label="prediction")
plt.legend()